In [ ]:
from commonfunctions import *
from skimage import io
from skimage.color import rgba2rgb, rgb2gray
from skimage import filters
from skimage.feature import hog
import numpy as np
import cv2
import sys
import os
from PHOG_V2 import *
from Faces import *
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn import svm
import pickle

from sklearn.datasets import fetch_olivetti_faces

In [ ]:
def getFeatureArray(size = 24):
    feature_array = []
    loop_on_array = np.zeros((size,size))
    feature_names = ["Edge_Horizontal","Edge_Vertical","Line_Horizontal","Line_Verticle","Rectangle"]
    for name in feature_names:
                
        if name == "Edge_Horizontal":
            width = 1
            height = 0
            while not(width >= size and height >= size):
                for i in range(size - width):
                    for j in range(size - height):
#                         print((i,j),(i + width,j + height))
                        feature_array.append([(i,j),(i + width,j + height),name])
#                 print(len(feature_array), width, height)
#                 input()
                if width < size:
                    width += 2
                elif height != size:
                    width = 1
                    height += 1

        elif name == "Edge_Vertical":
            width = 0
            height = 1
            while not(width >= size and height >= size):
                for i in range(size - width):
                    for j in range(size - height):
#                         print((i,j),(i + width,j + height))
                        feature_array.append([(i,j),(i + width,j + height),name])
                if height < size:
                    height += 2
                elif width != size:
                    height = 1
                    width += 1

        elif name == "Line_Horizontal":
            width = 2
            height = 0
            while not(width >= size and height >= size):
                for i in range(size - width):
                    for j in range(size - height):
#                         print((i,j),(i + width,j + height))
                        feature_array.append([(i,j),(i + width,j + height),name])
                if width < size:
                    width += 3
                elif height != size:
                    width = 2
                    height += 1     
                    
        elif name == "Line_Verticle":
            width = 0
            height = 2
            while not(width >= size and height >= size):
                for i in range(size - width):
                    for j in range(size - height):
#                         print((i,j),(i + width,j + height))
                        feature_array.append([(i,j),(i + width,j + height),name])
                if height < size:
                    height += 3
                elif width != size:
                    height = 2
                    width += 1  

        elif name == "Rectangle":
            width = 1
            height = 1
            while not(width >= size and height >= size):
                for i in range(size - width):
                    for j in range(size - height):
                        feature_array.append([(i,j),(i + width,j + height),name])
                if width < size:
                    width += 2
                elif height != size:
                    width = 1
                    height += 2

    return feature_array

feature_array = getFeatureArray(24)
print(len(feature_array))

In [ ]:
def preprocessImg(img):
    img_copy = np.copy(img)
    if len(img_copy.shape) != 2:
        img_copy = rgb2gray(img_copy)
    img_copy = cv2.resize(img_copy, dsize=(24, 24), interpolation=cv2.INTER_CUBIC) 
    return img_copy

In [ ]:
def getIntegralImage(img):
    new_img = np.zeros(img.shape)
#     new_img[i,j] = img[0,0]
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            new_img[i,j] = np.sum(img[0:i+1,0:j+1])
    return new_img

In [ ]:
def calculateSumOfIntegralImage_V2(int_img,st_x,st_y,end_x,end_y):
        if (st_x-1 < 0) or (st_y-1) < 0:
            return int_img[end_x,end_y]
        if (end_x == 24):
            end_x -= 1
        if (end_y == 24):
            end_y -= 1
        if (st_x - 1 == 24):
            st_x -= 2
        if (st_y - 1 == 24):
            st_y -= 2
        # print(int_img.shape,st_x,st_y,end_x,end_y)
        return int_img[end_x,end_y] + int_img[st_x-1,st_y-1] - int_img[end_x,st_y-1] - int_img[st_x-1,end_y]

In [ ]:
def harrEdgeHorizontal(int_img,feature_params, scale_factor):
    # print(feature_params)
    st_x, st_y = feature_params[0] 
    end_x, end_y = feature_params[1] 
    st_x = int(st_x * scale_factor)
    st_y = int(st_y * scale_factor)
    end_x = int(end_x * scale_factor)
    end_y = int(end_y * scale_factor)
    middle_x = (end_x + st_x)//2
    middle_y = (end_y + st_y)//2
    up_value = calculateSumOfIntegralImage_V2(int_img,st_x,st_y,middle_x,end_y)
    down_value = calculateSumOfIntegralImage_V2(int_img,middle_x + 1,st_y,end_x,end_y)
    return up_value - down_value

In [ ]:
def harrEdgeVerticle(int_img,feature_params, scale_factor):
    # print(feature_params)
    st_x, st_y = feature_params[0] 
    end_x, end_y = feature_params[1] 
    st_x = int(st_x * scale_factor)
    st_y = int(st_y * scale_factor)
    end_x = int(end_x * scale_factor)
    end_y = int(end_y * scale_factor)
    middle_x = (end_x + st_x)//2
    middle_y = (end_y + st_y)//2
    left_value = calculateSumOfIntegralImage_V2(int_img,st_x,st_y,end_x,middle_y)
    right_value = calculateSumOfIntegralImage_V2(int_img,st_x,middle_y + 1,end_x,end_y)
    return left_value - right_value

In [ ]:
def harrLineHorizontal(int_img,feature_params, scale_factor):
    # print(feature_params)
    st_x, st_y = feature_params[0]
    end_x, end_y = feature_params[1] 
    st_x = int(st_x * scale_factor)
    st_y = int(st_y * scale_factor)
    end_x = int(end_x * scale_factor)
    end_y = int(end_y * scale_factor)
    third_x = (end_x + st_x + st_x)//3
    third_2x = (end_x + end_x + st_x)//3
    third_y = (end_y + st_y + st_y)//3
    third_2y = (end_y + end_y + st_y)//3
#     print("st_x,st_y,third_x,third_y,third_2x,third_2y,end_x,end_y")
#     print(st_x,st_y,third_x,third_y,third_2x,third_2y,end_x,end_y)
    up_value = calculateSumOfIntegralImage_V2(int_img,st_x,st_y,third_x,end_y)
    middle_value = calculateSumOfIntegralImage_V2(int_img,third_x + 1,st_y,third_2x,end_y)
    down_value = calculateSumOfIntegralImage_V2(int_img,third_2x + 1,st_y,end_x,end_y)
    return up_value + down_value - middle_value

In [ ]:
def harrLineVerticle(int_img,feature_params, scale_factor):
    # print(feature_params)
    st_x, st_y = feature_params[0] 
    end_x, end_y = feature_params[1] 
    st_x = int(st_x * scale_factor)
    st_y = int(st_y * scale_factor)
    end_x = int(end_x * scale_factor)
    end_y = int(end_y * scale_factor)
    third_x = (end_x + st_x + st_x)//3
    third_2x = (end_x + end_x + st_x)//3
    third_y = (end_y + st_y + st_y)//3
    third_2y = (end_y + end_y + st_y)//3
    left_value = calculateSumOfIntegralImage_V2(int_img,st_x,st_y,end_x,third_y)
    middle_value = calculateSumOfIntegralImage_V2(int_img,st_x,third_y + 1,end_x,third_2y)
    right_value = calculateSumOfIntegralImage_V2(int_img,st_x,third_2y + 1,end_x,end_y)
    return left_value + right_value - middle_value

In [ ]:
def harrRectangle(int_img,feature_params,scale_factor):
    # print(feature_params)
    # try:
        st_x, st_y = feature_params[0]
        end_x, end_y = feature_params[1] 
        st_x = int(st_x * scale_factor)
        st_y = int(st_y * scale_factor)
        end_x = int(end_x * scale_factor)
        end_y = int(end_y * scale_factor)
        middle_x = (end_x + st_x)//2
        middle_y = (end_y + st_y)//2
        up_left_value = calculateSumOfIntegralImage_V2(int_img,st_x,st_y,middle_x,middle_y)
        up_right_value = calculateSumOfIntegralImage_V2(int_img,st_x,middle_y + 1,middle_x,end_y)
        down_left_value = calculateSumOfIntegralImage_V2(int_img,middle_x + 1,st_y,end_x,middle_y)
        down_right_value = calculateSumOfIntegralImage_V2(int_img,middle_x + 1,middle_y + 1,end_x,end_y)
#         print(up_left_value,up_right_value,down_left_value,down_right_value,up_left_value + down_right_value - up_right_value - down_left_value)
        return up_left_value + down_right_value - up_right_value - down_left_value
    # except:
        # print(feature_params)

In [ ]:
def calculateHaar(int_img,feature_params, scale_factor):
    # try:
        if feature_params[2] == "Edge_Horizontal":
            return harrEdgeHorizontal(int_img,feature_params[0:2],scale_factor)
        elif feature_params[2] == "Edge_Vertical":
            return harrEdgeVerticle(int_img,feature_params[0:2],scale_factor)
        elif feature_params[2] == "Line_Horizontal":
            return harrLineHorizontal(int_img,feature_params[0:2],scale_factor)
        elif feature_params[2] == "Line_Verticle":
            return harrLineVerticle(int_img,feature_params[0:2],scale_factor)
        elif feature_params[2] == "Rectangle":
            return harrRectangle(int_img,feature_params[0:2],scale_factor)
    # except:
        # print(feature_params)
        # print(feature_params[0] * scale_factor)

In [ ]:
# stage # feature # values
learned = np.zeros((100,10,4))
count = 10
for i in range(100):
    with open('./Trained/filename'+ str(count) +'.txt') as f:
        lines = f.readlines()
        for j in range(10):
            learned[i][j][0] = int(lines[j * 4])
            learned[i][j][1] = float(lines[j * 4 + 1])
            learned[i][j][2] = float(lines[j * 4 + 2])
            learned[i][j][3] = int(lines[j * 4 + 3]) 
        count += 1

In [ ]:
print(learned[0])
print(feature_array[int(learned[0][0][0])])
print(learned[1][0][3] == False)

In [ ]:
def predictStamb(int_img,featue,thrs,more,feature_array,scale):
    try:
        res = calculateHaar(int_img,feature_array[int(featue)],scale)
        if (res >= thrs and more == True) or (res < thrs and more == False):
            return 1
        else:
            return 0
    except:
        print(int_img.shape)
        print(feature_array[int(featue)])
        print(scale)
        print(calculateHaar(int_img,feature_array[int(featue)],scale))
    #     print(int_img.shape)
    #     print(feature_array[int(featue)])
    #     print(scale)
    #     print(calculateHaar(int_img,feature_array[int(featue)],scale))

In [ ]:
def predictAdaBoost(int_img,learned,index,feature_array,scale):
    preds = []
    for i in range(learned[index].shape[0]):
        preds.append(learned[index][i][1] * predictStamb(int_img,learned[index][i][0],learned[index][i][2],learned[index][i][3],feature_array,scale))  
    pred = np.sum(preds)
    pred = np.sign(pred)
    return pred  

In [ ]:
# test_img = io.imread("./images/1.jpg")
# test_img = rgb2gray(test_img)
# show_images([test_img])
# print(test_img.shape)

In [ ]:
def slideWindow(test_img, learned, feature_array):
    winX = 24
    winY = 24
    scale = 1
    results = []
    score = -9999999999999
    while (winX < test_img.shape[0]) and (winY < test_img.shape[1]): 
        # print(winX,winY)
        for i in range(0,int(test_img.shape[0] - winX),1):
            for j in range(0,int(test_img.shape[1] - winY),1):
                # print(i , int(i+winX) ,j ,int(j + winY))
                if(i + winX > test_img.shape[0] or j + winY > test_img.shape[1]):
                    break
                croppedImg = np.copy(test_img[i:int(i+winX),j:int(j+winY)])
                int_img = getIntegralImage(croppedImg)
                val = 0
                for i in range(0,learned.shape[0]):
                    val += predictAdaBoost(int_img,learned,i,feature_array,scale)
                    # print(i,val)
                # print(val)
                if val >= score:
                    if val != score:
                        results = []
                    score = val
                    results.append(croppedImg)
        scale = scale * 1.25
        winX *= 1.25
        winY *= 1.25
        # print("I am here")
    return results

In [ ]:
# ress = slideWindow(test_img,learned,feature_array)

In [ ]:
# print(len(ress))
# show_images(ress[23780:23791])

In [ ]:
# test_img2 = cv2.resize(test_img, dsize=(24, 24), interpolation=cv2.INTER_CUBIC) 

# show_images([test_img2])

# for i in range(learned.shape[0]):
#     print(i,predictAdaBoost(getIntegralImage(test_img2),learned,i,feature_array,1))

In [ ]:
bunch = fetch_olivetti_faces(data_home=None, shuffle = False, return_X_y = False)
# show_images(bunch.images[0:10])

def ReadFile(path,label,imgs,labels):
    files = os.listdir(path)
    num_images = len(files)
    count = 0
    for f in files:
        count += 1
        if count >= 300 and count < 400:
            img = io.imread(os.path.join(path, f), as_gray=False)
            img= preprocessImg(img)
            imgs.append(img)
            labels.append(label)
            # return 

path = "./FaceDetection/False/NotHuman"
imgs = []
labels = []
ReadFile(path,0,imgs,labels)
print(len(imgs))
print(len(labels))
show_images(imgs[0:10])

In [ ]:
data = np.copy(bunch.images)
count = 0
for i in range (data.shape[0]):
    count += 1
    if count >= 300 and count < 400:
        imgs.append(preprocessImg(data[i]))
        labels.append(1)

print(len(imgs))
print(len(labels))
print(labels)

In [ ]:
sum = 0
for i in range(len(imgs)):
    res = 1
    # show_images([imgs[i]])
    int_img = getIntegralImage(imgs[i])
    for i in range(learned.shape[0]):
        res = res * predictAdaBoost(int_img,learned,i,feature_array,1)
    if (res == labels[i]):
        sum = sum + 1
print(sum)
print(sum / len(labels))


In [ ]:
# show_images([bunch.images[0]])

# resBunch = slideWindow(bunch.images[0],learned,feature_array)

In [ ]:
# print(len(resBunch))
# show_images(resBunch)

In [ ]:
test_img = io.imread("p1.png")
# test_img = rgb2gray(test_img)
show_images([test_img])
print(test_img.shape)

In [ ]:
# result = slideWindow(test_img,learned,feature_array)

In [ ]:
# print(len(result))
# show_images(result)

In [ ]:
test_img2 = cv2.resize(test_img, dsize=(int(test_img.shape[0] / 8), int(test_img.shape[1] / 8)), interpolation=cv2.INTER_CUBIC) 
show_images([test_img2])
print(test_img2.shape)

In [ ]:
result = slideWindow(test_img2,learned,feature_array)

In [ ]:
print(len(result))
show_images(result)

In [ ]:
test_img3 = io.imread("p2.png")
# test_img = rgb2gray(test_img)
show_images([test_img3])
print(test_img3.shape)

In [ ]:
test_img4 = cv2.resize(test_img3, dsize=(int(test_img3.shape[0] / 8), int(test_img3.shape[1] / 8)), interpolation=cv2.INTER_CUBIC) 
show_images([test_img4])
print(test_img4.shape)

In [ ]:
result = slideWindow(test_img4,learned,feature_array)

In [ ]:
print(len(result))
show_images(result)

In [ ]:
# test_img3 = io.imread("3.jpg")
# test_img3 = rgb2gray(test_img3)
# show_images([test_img3])
# print(test_img3.shape)

In [ ]:
# test_img4 = cv2.resize(test_img3, dsize=(int(test_img3.shape[0] / 2), int(test_img3.shape[1] / 2)), interpolation=cv2.INTER_CUBIC) 
# show_images([test_img4])
# print(test_img4.shape)

In [ ]:
# result = slideWindow(test_img4,learned,feature_array)

In [ ]:
# print(len(result))
# show_images(result[0:10])

In [ ]:
test_img3 = io.imread("4.jpg")
test_img3 = rgb2gray(test_img3)
show_images([test_img3])
print(test_img3.shape)

In [ ]:
test_img4 = cv2.resize(test_img3, dsize=(int(test_img3.shape[0] / 3), int(test_img3.shape[1] / 3)), interpolation=cv2.INTER_CUBIC) 
show_images([test_img4])
print(test_img4.shape)

In [ ]:
result = slideWindow(test_img4,learned,feature_array)

In [ ]:
print(len(result))
show_images(result)